In [1]:
import pandas as pd
import numpy as np
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="white")
sns.set(style="whitegrid", color_codes=True)
## Set random seed for reproducibility
np.random.seed(1234)

In [3]:
credit = pd.read_csv("../datafiles/credit_data.csv")
print(credit.shape)

(4454, 14)


In [5]:
credit = credit.dropna()

In [7]:
numerical_features = ['Seniority', 'Time', 'Age', 'Expenses', 'Income', 
                      'Assets', 'Debt', 'Amount', 'Price']
categorical_features = ['Home', 'Marital', 'Records', 'Job']

In [8]:
credit_ohe = pd.get_dummies(credit[categorical_features], drop_first = True)

In [9]:
credit_num = credit[numerical_features]
credit_num = (credit_num - credit_num.min()) / (credit_num.max() - credit_num.min())

In [10]:
X = pd.concat([credit[numerical_features], credit_ohe], axis = 1)
X

,Seniority,Time,Age,Expenses,Income,Assets,Debt,Amount,Price,Home_other,...,Home_priv,Home_rent,Marital_married,Marital_separated,Marital_single,Marital_widow,Records_yes,Job_freelance,Job_others,Job_partime
0,9,60,30,73,129.0,0.0,0.0,800,846,0,...,0,1,1,0,0,0,0,1,0,0
1,17,60,58,48,131.0,0.0,0.0,1000,1658,0,...,0,1,0,0,0,1,0,0,0,0
2,10,36,46,90,200.0,3000.0,0.0,2000,2985,0,...,0,0,1,0,0,0,1,1,0,0
3,0,60,24,63,182.0,2500.0,0.0,900,1325,0,...,0,1,0,0,1,0,0,0,0,0
4,0,36,26,46,107.0,0.0,0.0,310,910,0,...,0,1,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4449,1,60,39,69,92.0,0.0,0.0,900,1020,0,...,0,1,1,0,0,0,0,0,0,0
4450,22,60,46,60,75.0,3000.0,600.0,950,1263,0,...,0,0,1,0,0,0,0,0,0,0
4451,0,24,37,60,90.0,3500.0,0.0,500,963,0,...,0,0,1,0,0,0,0,0,0,1
4452,0,48,23,49,140.0,0.0,0.0,550,550,0,...,0,1,0,0,1,0,0,1,0,0


In [12]:
from sklearn.model_selection import train_test_split

from sklearn.model_selection import cross_val_score, KFold

In [13]:
y = credit['Status']
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    train_size = 0.8)

In [14]:
from sklearn.neural_network import MLPClassifier

nn_credit = MLPClassifier(hidden_layer_sizes=(6,3),  
                      activation='logistic', 
                      solver='lbfgs', 
                      max_iter = 1000,
                     random_state = 1)

In [15]:
nn_credit.fit(X_train, y_train)
y_test_pred = nn_credit.predict_proba(X_test)

/Users/u0784726/miniforge3/envs/geog5160/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:541: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


In [16]:
from sklearn.metrics import roc_auc_score
print('AUROC: %.2f'
      % roc_auc_score(y_test, y_test_pred[:,1]))

AUROC: 0.68


In [18]:
from sklearn.model_selection import GridSearchCV
nnet_param = {'hidden_layer_sizes':[500,1000,1500,2000],
             'max_iter':[500,1000,1500,2000],
             'learning_rate_init': [1e-5, 1e-4, 1e-3]}
nn_reg_tuned = GridSearchCV(MLPClassifier(), nnet_param, 
                             scoring='roc_auc', cv=5)
nn_reg_tuned.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=MLPClassifier(),
             param_grid={'hidden_layer_sizes': [500, 1000, 1500, 2000],
                         'learning_rate_init': [1e-05, 0.0001, 0.001],
                         'max_iter': [500, 1000, 1500, 2000]},
             scoring='roc_auc')